# Least squares reduction

In [1]:
import numpy as np
import glob
import matplotlib
import matplotlib.pyplot as plt

matplotlib.rcParams['image.origin'] = 'lower'
matplotlib.rcParams['image.interpolation'] = 'nearest'


In [1]:
import sys
codefolder = '../../../../crispy'
if codefolder not in sys.path: sys.path.append(codefolder)
from crispy.tools.initLogger import getLogger
log = getLogger('crispy')
from crispy.params import Params
codefolder = '../../../crispy'
par = Params(codefolder)

### This is a test wavecal with slightly different PSFLet models

In [3]:
par.wavecalDir = par.prefix+'/wavecalR50_770_test/'
par.hdr


SIMPLE  =                    T / conforms to FITS standard                      
BITPIX  =                    8 / array data type                                
NAXIS   =                    0 / number of array dimensions                     
EXTEND  =                    T                                                  
COMMENT                                                                         
COMMENT ************************************************************            
COMMENT ********************** General parameters ******************            
COMMENT ************************************************************            
COMMENT                                                                         
NLENS   =                  108 / # lenslets across array                        
PITCH   =             0.000174 / Lenslet pitch (meters)                         
INTERLAC=                    2 / Interlacing                                    
PHILENS =    26.565051177077

### create flatfield if needed

In [4]:
from crispy.unitTests import testCreateFlatfield
# par.savePoly = True
par.saveRotatedInput = True
testCreateFlatfield(par)

crispy - INFO - Reduced cube will have 44 wavelength bins
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming endpoints wavelist is given
[ 700.          703.18181818  706.36363636  709.54545455  712.72727273
  715.90909091  719.09090909  722.27272727  725.45454545  728.63636364
  731.81818182  735.          738.18181818  741.36363636  744.54545455
  747.72727273  750.90909091  754.09090909  757.27272727  760.45454545
  763.63636364  766.81818182  770.          773.18181818  776.36363636
  779.54545455  782.72727273  785.90909091  789.09090909  792.27272727
  795.45454545  798.63636364  801.81818182  805.          808.18181818
  811.36363636  814.54545455  817.72727273  820.90909091  824.09090909
  827.27272727  830.45454545  833.63636364  836.81818182  840.        ]


crispy - INFO - Writing data to ../../../crispy/SimResults/imagePlaneRot.fits
crispy - INFO - Number of detector pixels per lenslet: 13.384615
crispy - INFO - Rebinning final detector. Image has dimensions 1024x1024
crispy - INFO - Done.
crispy - INFO - Performance: 44 seconds total
crispy - INFO - Writing data to ../../../crispy/unitTestsOutputs/flatfield.fits


In [2]:
from crispy.IFS import reduceIFSMap
#tmp=reduceIFSMap(par,par.wavecalDir+'det_760.fits',method='lstsq')
# tmp=reduceIFSMap(par,par.wavecalDir+'polychromeR50stack_highdensity.fits',method='lstsq')
# tmp=reduceIFSMap(par,par.unitTestsOutputs+'/flatfield.fits')
#tmp=reduceIFSMap(par,par.unitTestsOutputs+'/flatfield.fits',method='lstsq')
tmp=reduceIFSMap(par,par.unitTestsOutputs+'/flatfield.fits',method='optext')
# tmp=reduceIFSMap(par,par.wavecalDir+'polychromeR50stack.fits',method='lstsq')


crispy - INFO - Read data from HDU 1 of ../../../crispy/unitTestsOutputs/flatfield.fits
crispy - INFO - Reduced cube will have 18 wavelength bins
crispy - INFO - Writing data to ../../../crispy/SimResults/flatfield_red_optext.fits
crispy - INFO - Elapsed time: 13.112292s


In [6]:
from crispy.tools.image import Image
from crispy.tools.reduction import calculateWaveList
import astropy.units as u
offaxis = Image(filename='../../../../OS5/offaxis/spc_offaxis_psf.fits')
lamc=770.
BW=0.17
offaxis.header['LAM_C'] = lamc/1000.
from crispy.IFS import polychromeIFS
Nlam = offaxis.data.shape[0]
lamlist = lamc*np.linspace(1.-BW/2.,1.+BW/2.,Nlam)
lam_midpts,lam_endpts = calculateWaveList(par,Nspec=45)
par.saveDetector=True
polychromeIFS(par,lam_midpts,offaxis,parallel=True,wavelist_endpts=lam_endpts)

crispy - INFO - Read data from HDU 0 of ../../../../OS5/offaxis/spc_offaxis_psf.fits
crispy - INFO - Reduced cube will have 44 wavelength bins
crispy - ERROR - Number of wavelengths does not match the number of slices
crispy - INFO - Using PSFlet gaussian approximation
crispy - INFO - Assuming endpoints wavelist is given
[ 700.          703.18181818  706.36363636  709.54545455  712.72727273
  715.90909091  719.09090909  722.27272727  725.45454545  728.63636364
  731.81818182  735.          738.18181818  741.36363636  744.54545455
  747.72727273  750.90909091  754.09090909  757.27272727  760.45454545
  763.63636364  766.81818182  770.          773.18181818  776.36363636
  779.54545455  782.72727273  785.90909091  789.09090909  792.27272727
  795.45454545  798.63636364  801.81818182  805.          808.18181818
  811.36363636  814.54545455  817.72727273  820.90909091  824.09090909
  827.27272727  830.45454545  833.63636364  836.81818182  840.        ]
crispy - INFO - Writing data to ../..

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [7]:
from crispy.IFS import reduceIFSMap
tmp=reduceIFSMap(par,par.exportDir+'/detectorFrame.fits',method='lstsq')


crispy - INFO - Read data from HDU 1 of ../../../crispy/SimResults/detectorFrame.fits
crispy - INFO - Reduced cube will have 10 wavelength bins
crispy - INFO - Elapsed time: 12.421115s


In [8]:
resid = Image(par.exportDir+'/detectorFrame_red_lstsq_resid.fits')
detframe=Image(par.exportDir+'/detectorFrame.fits')
ratio = resid.data/detframe.data
ratio[ratio>=1]=0.
ratio[ratio<=-1]=0.0
Image(data=ratio).write(par.exportDir+'/resid_detectorFrame_ratio.fits')

crispy - INFO - Read data from HDU 0 of ../../../crispy/SimResults/detectorFrame_red_lstsq_resid.fits
crispy - INFO - Read data from HDU 1 of ../../../crispy/SimResults/detectorFrame.fits
crispy - INFO - Writing data to ../../../crispy/SimResults/resid_detectorFrame_ratio.fits


/Users/mrizzo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in divide
  app.launch_new_instance()
/Users/mrizzo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: RuntimeWarning: invalid value encountered in divide
  app.launch_new_instance()
/Users/mrizzo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in greater_equal
/Users/mrizzo/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: RuntimeWarning: invalid value encountered in less_equal


In [9]:
from crispy.IFS import reduceIFSMap
tmp=reduceIFSMap(par,par.wavecalDir+'/det_750.fits',method='lstsq')


crispy - INFO - Read data from HDU 1 of ../../../crispy/ReferenceFiles/wavecalR50_770_test//det_750.fits
crispy - INFO - Reduced cube will have 10 wavelength bins
crispy - INFO - Elapsed time: 12.673125s
